In [33]:
# Load data from MySQL to perform exploratory data analysis
import settings
import mysql.connector
import pandas as pd
import time

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
#%matplotlib inline
# Using plotly.express
import plotly.express as px
import datetime
from IPython.display import clear_output


In [22]:
'''
This plot shows the latest Twitter data within 30 mins and will automatically update.
In polarity:
    -1 means negative
    0  means neural
    +1 means positive
'''
while True:
    clear_output()
    db_connection = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="password",
        database="TwitterDB",
        charset = 'utf8'
    )
    # Load data from MySQL
    timenow = (datetime.datetime.utcnow() - datetime.timedelta(hours=0, minutes=10)).strftime('%Y-%m-%d %H:%M:%S')
    query = "SELECT id_str, text, created_at, polarity, user_location FROM {} WHERE created_at >= '{}' " \
                     .format(settings.TABLE_NAME, timenow)
    df = pd.read_sql(query, con=db_connection)
    # UTC for date time at default
    df['created_at'] = pd.to_datetime(df['created_at'])

    # Clean and transform data to enable time series
    result = df.groupby([pd.Grouper(key='created_at', freq='2s'), 'polarity']).count().unstack(fill_value=0).stack().reset_index()
    #result['created_at'] = pd.to_datetime(result['created_at']).apply(lambda x: x.strftime('%m-%d %H:%M'))
    #result = df
    result = result.rename(columns={"id_str": "Num of '{}' mentions".format(settings.TRACK_WORDS[0]), "created_at":"Time in UTC"})
    
    fig = px.line(result, x='Time in UTC', y="Num of '{}' mentions".format(settings.TRACK_WORDS[0]), color='polarity')
    fig.show()
    time.sleep(60)

KeyboardInterrupt: 

In [44]:
import re
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="password",
    database="TwitterDB",
    charset = 'utf8'
)
query = "SELECT id_str, text, created_at, polarity, user_location FROM {}  ".format(settings.TABLE_NAME)
df = pd.read_sql(query, con=db_connection)
# UTC for date time at default
df['created_at'] = pd.to_datetime(df['created_at'])
# 
'''
text = df.loc[:,['text']]
text['text'] = text['text'].str.replace('[^a-zA-Z]', ' ')
text['text'] = text['text'].str.replace('RT ', ' ')
text['text'] = text['text'].str.lower()
text['char_counts'] = text['text'].str.len()
text['word_counts'] = text['text'].str.split().str.len()
text['avg_word_len'] = text['char_counts'] / text['word_counts']
'''

content = ' '.join(df["text"])
content = re.sub(r"http\S+", "", content)
content = content.replace('RT ', ' ').replace('&amp;', 'and')
content = re.sub('[^A-Za-z0-9]+', ' ', content)
content = content.lower()

tokenized_word = word_tokenize(content)
stop_words=set(stopwords.words("english"))
filtered_sent=[]
for w in tokenized_word:
    if w not in stop_words:
        filtered_sent.append(w)
#print("Filterd Sentence:",filtered_sent)

fdist = FreqDist(filtered_sent)
fd = pd.DataFrame(fdist.most_common(25), columns = ["Word","Frequency"]).drop([0]).reindex()
# Plot Bar chart
fig = px.bar(fd, x="Word", y="Frequency")
fig.show()